In [100]:
#import files
import requests
from datetime import date
from datetime import datetime
import json

In [102]:
# derive the appropriate exchange rate for the currency code provided
def calc_exchange_rate(currency_code):
    global calc_out
    global exchange_date
    global exchange_rates
    global run_date
    
    # restore exchange date, dictionary of exchange rates, and last run date of the API
    %store -r exchange_date
    %store -r exchange_rates
    %store -r run_date
    
    # if API was already invoked today, use the existing dictionary of exchange rates
    # (we're using a free API which only updates once\day)
    if (run_date == date.today()):
        calc_out = exchange_rates[currency_code]
    else:
        # invoke function to call API
        calc_out = get_curr_exchg_rate(currency_code)
        run_date = date.today()
    
    # store the exchange rate for the given currency code, along with run date
    %store calc_out
    %store run_date
    

In [104]:
# call api to get dictionary of exchange rates
def get_curr_exchg_rate(currency_code):
    global response_data
    global exchg_rate_data
    global exchange_date
    global exchange_rates
    
    # call API
    url = 'https://open.er-api.com/v6/latest/USD'
    response_data = requests.get(url)
    %store response_data
    print(response_data)
    
    # extract and format data
    exchg_rate_data = response_data.json()
    %store exchg_rate_data
    print(json.dumps(exchg_rate_data, indent=4))
    
    # check return code; if 'success', extract the exchange data and dictionary of exchange rates
    resp_cd = exchg_rate_data["result"]
    if resp_cd == "success":
        date_object = datetime.strptime(exchg_rate_data['time_last_update_utc'], '%a, %d %b %Y %H:%M:%S %z')
        exchange_date = date_object.strftime('%Y-%m-%d')
        %store exchange_date
        exchange_rates = exchg_rate_data['rates']
        %store exchange_rates
        return exchange_rates[currency_code]
    else:
        return 0
    
    # thank you to https://www.exchangerate-api.com/docs/free for providing the API